# 利用 Paddle Quantum 的 qchem 模块进行量子化学计算
_Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved._


qchem 是基于 Paddle Quantum 推出的用于量子化学研究的工具集。qchem 为量子化学领域的研究者提供了一系列工具，使他们可以利用量子计算方法完成量子化学任务。与此同时，qchem 也提供了方便开发者进行功能拓展的方式。目前，qchem 正处于开发之中，您可以将需求和建议通过 Github 的 issue 或 pull request 反馈给我们，我们会及时给出回复。

## 分子基态能量计算
qchem 提供了 `run_chem` 函数来计算给定分子的基态能量和基态波函数。让我们通过计算氢分子基态能量的例子来了解一下整个计算过程。首先，让我们导入需要用的函数。

In [ ]:
from paddle_quantum import qchem

接下来，让我们把与计算相关的信息输入到 `run_chem` 函数中，这些信息包括：分子的几何结构、分子的电荷、计算需要用到的量子化学基函数等。

In [ ]:
# define the geometry of hydrogen molecule, length unit use angstrom.
h2_geometry = [("H", [0.0, 0.0, 0.0]), ("H", [0.0, 0.0, 0.74])]
charge = 0
basis_set = "sto-3g"

现在，我们需要选择一种多体波函数的量子线路模版来作为基态能量计算中的变分波函数。目前，qchem 提供了两种模式，分别是 "hardware efficient" [<sup>1</sup>](#refer-1) 和 "hartree fock" [<sup>2</sup>](#refer-2) ，更多的模式正在开发中，很快也会上线。

In [ ]:
# call run_chem function with "hardware efficient" ansatz.
h2_gs_en, h2_wf_model = qchem.run_chem(
    h2_geometry,
    "hardware efficient",
    basis_set, 
    charge
)

# additional information for optimizer can be passed using `optimizer_option` keyword argument.
h2_gs_en, h2_wf_model = qchem.run_chem(
    h2_geometry,
    "hardware efficient",
    basis_set, 
    charge,
    optimizer_option={"learning_rate": 0.6, "weight_decay": 0.9}
)

# additional information for ansatz can be passed using `ansatz_option` keyword argument, e.g.
# "hardware efficient" ansatz has a parameter "cir_depth", which can be used to specify the depth
# of quantum circuit.
h2_gs_en, h2_wf_model = qchem.run_chem(
    h2_geometry,
    "hardware efficient",
    basis_set, 
    charge,
    ansatz_option={"cir_depth": 5}
)

除了上面这些参数，我们也可以通过给定 `max_iters` 和 `a_tol` 来控制训练过程的迭代次数和收敛条件。如果想使用 "hartree fock" 量子线路，我们只需要把上面代码中的 "hardware efficient" 替换为 "hartree fock" 就可以了。我们也可以通过 `print(h2_wf_model.circuit)` 的方法来查看氢分子波函数的量子线路。

## 自定义量子线路
在 qchem 中，我们也为感兴趣的使用者提供了自定义量子线路的方法。我们只需要继承 qchem 中的 Qmodel 类就可以像在 paddlepaddle 中定义神经网络一样定义波函数对应的量子线路。

In [ ]:
from paddle_quantum.circuit import UAnsatz
from paddle_quantum import qchem
from paddle_quantum.qchem.layers import CrossResonanceLayer, EulerRotationLayer


# Your own model should inherit from `Qmodel`.
## NOTE: THIS MODEL IS ONLY DEFINED FOR DEMONSTRATION PURPOSE! 
class MyAnsatz(qchem.QModel):
    def __init__(self, n_qubits):
        super().__init__(n_qubits)

        self.entangle = CrossResonanceLayer(self._n_qubit)
        self.rot = EulerRotationLayer(self._n_qubit)

    def forward(self, state):
        self._circuit = UAnsatz(self.n_qubit)

        out = self.entangle(state)
        self._circuit += self.entangle.circuit

        out = self.rot(out)
        self._circuit += self.rot.circuit

        out = self.entangle(out)
        self._circuit += self.entangle.circuit

        return out

# instantiate your model
my_cir = MyAnsatz(5)

在定义好线路之后，我们就可以利用 paddlepaddle 提供的多种优化器来优化线路中的参数以获得最佳结果。

In [ ]:
# use paddlepaddle's optimizer
import numpy as np 
import paddle

optimizer = paddle.optimizer.Adam(parameters=my_cir.parameters(), learning_rate=0.08)

# define the loss function
## NOTE: THIS LOSS FUNCTION IS ONLY DEFINED FOR DEMONSTRATION PURPOSE!
def loss_fn(state: paddle.Tensor) -> paddle.Tensor:
    return paddle.norm(state.real())

# start learning
s0 = np.zeros((2**5,), dtype=np.complex128)
s0[0] = 1.0+0.0j
s0 = paddle.to_tensor(s0)

for i in range(10):
    loss = loss_fn(my_cir(s0))
    print(f"At {i:>d}th step: loss={loss.item():>.5f}.")

    optimizer.clear_grad()
    loss.backward()
    optimizer.step()

---
## 参考文献

[1] Kandala, Abhinav, et al. "Hardware-efficient variational quantum eigensolver for small molecules and quantum magnets." [Nature 549.7671 (2017): 242-246.](https://www.nature.com/articles/nature23879)

[2] Arute, Frank, et al. "Hartree-Fock on a superconducting qubit quantum computer." [Science 369.6507 (2020): 1084-1089.](https://www.science.org/doi/10.1126/science.abb9811)